In [1]:
import jax
import jax.numpy as jnp

ModuleNotFoundError: No module named 'jax'

In [ ]:
t = jnp.linspace(0, 20, 1000)

m = 1.
c = 0.1
k = 1.
y0 = 1.
y0_prime = 0.


# create a one layer neural network with `tanh` activation function
def nn(params, t):
    return params['W2'] @ jnp.tanh(params['W1'] * t + params['b1']) + params['b2']


initializer = jax.nn.initializers.glorot_uniform()
key = jax.random.PRNGKey(42)
*init_keys, key = jax.random.split(key, 5)

h = 30
params = {
    'W1': initializer(init_keys[0], (h, 1), jnp.float32)[:, 0],
    'W2': initializer(init_keys[1], (1, h), jnp.float32)[0, :],
    'b1': initializer(init_keys[2], (h, 1), jnp.float32)[:, 0],
    'b2': initializer(init_keys[3], (1, 1), jnp.float32)[0, 0],
}

nn(params, t[0])

We want to train the neural network for the ODE 

$$
m * \ddot y + c * \dot y + k * y = 0,
$$
with $m = 1, c = 0.1$ and $k = 1$. 

Therefore we define an objective 

$$
\mathcal{L}(\mathbf{W}) = \frac{1}{M}\sum_{i=1}^M (m * \ddot y_\mathbf{W}(t_i) + c * \dot y_\mathbf{W}(t_i) + k * y_\mathbf{W}(t_i)) ^ 2 + (y_\mathbf{W}(0) - y_0) ^ 2 + (\dot y_\mathbf{W}(0) - \dot y_0) ^ 2
$$

In [ ]:

def ode(params, t):
    _nn = lambda t: nn(params, t)
    dnn = jax.grad(_nn)
    ddnn = jax.grad(dnn)
    return m * ddnn(t) + c * dnn(t) + k * _nn(t)

def objective(params, ts):
    ys = jax.vmap(ode, (None, 0))(params, ts)
    ode_loss = jnp.mean(ys ** 2)
    init_loss1 = (nn(params, 0.) - y0) ** 2
    init_loss2 = (jax.grad(nn, 1)(params, 0.) - y0_prime) ** 2
    return ode_loss + init_loss1 + init_loss2

objective(params, t)


In [ ]:
import optax

def fit(params: optax.Params, optimizer: optax.GradientTransformation, key) -> optax.Params:
    opt_state = optimizer.init(params)

    @jax.jit
    def step(params, opt_state, batch):
        loss_value, grads = jax.value_and_grad(objective)(params, batch)
        updates, opt_state = optimizer.update(grads, opt_state, params)
        params = optax.apply_updates(params, updates)
        return params, opt_state, loss_value
    
    for epoch in range(5000):
        shuffle_key, key = jax.random.split(key)
        batches = jax.random.permutation(shuffle_key, t)
        batches = batches.reshape(10, -1)
        for batch in batches:
            params, opt_state, loss_value = step(params, opt_state, batch)
        if epoch % 100 == 0:
            print(f'epoch {epoch}, loss: {loss_value}')
    return params

# Finally, we can fit our parametrized function using the Adam optimizer
# provided by optax.
optimizer = optax.adam(learning_rate=1e-2)
train_key, key = jax.random.split(key)
params = fit(params, optimizer, key)


In [ ]:
# import jaxopt
# solver = jaxopt.LBFGS(fun=objective)
# params = solver.run(params, t).params

In [ ]:
def sol(t):
    r1 = - c / (2 * m) + 1 / (2*m) * (c ** 2 - 4 * k * m) ** (1 / 2)
    r2 = - c / (2 * m) - 1 / (2*m) * (c ** 2 - 4 * k * m) ** (1 / 2)
    c2 = r1 / (r1 - r2)
    c1 = 1 - c2
    return c1 * jnp.exp(r1 * t) + c2 * jnp.exp(r2 * t)


In [ ]:
import matplotlib.pyplot as plt

model = jax.vmap(lambda t: nn(params, t))

plt.plot(t, jnp.real(sol(t)), label='true_sol')
plt.plot(t, model(t), label='pred_sol')
plt.legend()